In [ ]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import librosa
import librosa.display
from IPython.display import Audio

In [ ]:
# Laddar datasetet och extraherar filvägar och etiketter från filnamnen
#labels = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
paths = []
labels = []

# Traversera alla filer i datasetets katalogstruktur
for dirname, _, filenames in os.walk(r'D:\DataSets\AudioWAV'):
  for filename in filenames:
    # Lagra fullständig filväg
    paths.append(os.path.join(dirname, filename))
    # Extrahera etikett från filnamnet
    label = filename.split('_')[-2]
    
    match label:
        case 'ANG':
          label = 'angry' 
        case 'DIS':
          label = 'disgust'
        case 'FEA':
          label = 'fear'
        case 'HAP':
          label = 'happy'
        case 'NEU':
          label = 'neutral'
        case 'SAD':
          label = 'sad'


    print(label)
    #label = label.split('.')[0]
    #print(label)
    labels.append(label.lower())
print('Dataset is loaded')


In [ ]:
## Skapar en DataFrame för att organisera filvägar och etiketter för vidare analys och modellträning

df = pd.DataFrame()
df['speech'] = paths
df['labels'] = labels

# Visar de första raderna i DataFrame för att bekräfta att data har laddats korrekt
df.head()

In [ ]:
# 
sns.countplot(df['labels'])

In [ ]:
# Visualiserar ljudvågsformen för en given ljuddata med emotion som titel
def waveplot(data, sr, emotion):
  plt.figure(figsize=(10,4))
  plt.title(emotion, size=20)
  librosa.display.waveshow(data, sr=sr)
  plt.show()

# Skapar ett spektrogram för en given ljuddata med emotion som titel
def spectogram(data, sr, emotion):
  x = librosa.stft(data) 
  xdb = librosa.amplitude_to_db(abs(x)) 
  plt.figure(figsize=(10,4))
  plt.title(emotion, size=20)
  librosa.display.specshow(xdb, sr=sr, x_axis='time', y_axis='hz')
  plt.colorbar()

In [ ]:
# kollar så att datan ser bra ut och se eventuella skillnader mellan känslor
emotion = 'fear'
path = np.array(df['speech'][df['labels'] == emotion])[0]
data, sampling_rate = librosa.load(path)
waveplot(data, sampling_rate, emotion)
spectogram(data, sampling_rate, emotion)
Audio(path)

In [ ]:
emotion = 'fear'
path = np.array(df['speech'][df['labels'] == emotion])[0]
print(path)

In [ ]:
# kollar så att datan ser bra ut och se eventuella skillnader mellan känslor
emotion = 'angry'
path = np.array(df['speech'][df['labels'] == emotion])[0]
data, sampling_rate = librosa.load(path)
waveplot(data, sampling_rate, emotion)
spectogram(data, sampling_rate, emotion)
Audio(path)

In [ ]:
# kollar så att datan ser bra ut och se eventuella skillnader mellan känslor
emotion = 'disgust'
path = np.array(df['speech'][df['labels'] == emotion])[0]
data, sampling_rate = librosa.load(path)
waveplot(data, sampling_rate, emotion)
spectogram(data, sampling_rate, emotion)
Audio(path)

In [ ]:
# kollar så att datan ser bra ut och se eventuella skillnader mellan känslor
emotion = 'neutral'
path = np.array(df['speech'][df['labels'] == emotion])[0]
data, sampling_rate = librosa.load(path)
waveplot(data, sampling_rate, emotion)
spectogram(data, sampling_rate, emotion)
Audio(path)

In [ ]:
# kollar så att datan ser bra ut och se eventuella skillnader mellan känslor
emotion = 'sad'
path = np.array(df['speech'][df['labels'] == emotion])[0]
data, sampling_rate = librosa.load(path)
waveplot(data, sampling_rate, emotion)
spectogram(data, sampling_rate, emotion)
Audio(path)

In [ ]:
# kollar så att datan ser bra ut och se eventuella skillnader mellan känslor
emotion = 'happy'
path = np.array(df['speech'][df['labels'] == emotion])[0]
data, sampling_rate = librosa.load(path)
waveplot(data, sampling_rate, emotion)
spectogram(data, sampling_rate, emotion)
Audio(path)

In [ ]:
# Konverterar ljudfilen till MFCC (Mel-Frequency Cepstral Coefficients) eftersom det är mer lämpligt för 
# att skapa en känsloigenkänningsmodell

def extract_mfcc(filename):
  y, sr = librosa.load(filename, duration=3, offset=0.5)
  mfcc = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T, axis=0)
  return mfcc

In [ ]:
# Checkar funktionen
extract_mfcc(df['speech'][0])

In [ ]:
# Extraherar MFCC (Mel-Frequency Cepstral Coefficients) för varje ljudfil
X_mfcc = df['speech'].apply(lambda x: extract_mfcc(x))

In [ ]:
# Ändrar datatypen till numpy array
X = [x for x in X_mfcc]
X = np.array(X)
X.shape

In [ ]:
type(X)

In [ ]:
## Lägger till en extra dimension till X för att uppfylla modellens inputkrav
X = np.expand_dims(X, -1)
X.shape


In [ ]:
# Konverterar alla labels till one-hot kodning för att möjliggöra klassificering av modellen
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
y = enc.fit_transform(df[['labels']])
y = y.toarray()

In [ ]:
# Ckeckar dimensioner
y.shape

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Bidirectional, BatchNormalization, Conv1D, MaxPooling1D, Flatten
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Skapar en sekventiell modell för att klassificera ljuddata
model = Sequential([
    # Första konvolutionslagret för att extrahera funktioner från tidsseriedata
    Conv1D(32, kernel_size=(3), activation='relu', input_shape=(40, 1)),  
    MaxPooling1D(pool_size=(2)),  # Minskar dimensionerna och behåller viktiga funktioner
    Dropout(0.3),  # Förhindrar överanpassning

    # Andra konvolutionslagret för att extrahera mer komplexa funktioner
    Conv1D(64, kernel_size=(3), activation='relu'),
    MaxPooling1D(pool_size=(2)),  # Minskar dimensionerna ytterligare
    Dropout(0.3),  # Förhindrar överanpassning

    Flatten(),  # Omvandlar data till en dimension för fullt anslutna lager

    # Första fullt anslutna lagret för att kombinera extraherade funktioner
    Dense(128, activation='relu'),
    Dropout(0.3),  # Förhindrar överanpassning
    BatchNormalization(),  # Normaliserar data för snabbare konvergens och stabilitet

    # Andra fullt anslutna lagret för ytterligare funktionkombination
    Dense(64, activation='relu'),
    Dropout(0.3),  # Förhindrar överanpassning
    BatchNormalization(),  # Normaliserar data

    # Tredje fullt anslutna lagret för ytterligare funktionkombination
    Dense(32, activation='relu'),
    Dropout(0.3),  # Förhindrar överanpassning

    # Utgångslager med softmax-aktivering för klassificering i 6 kategorier
    Dense(6, activation='softmax')
])

# Kompilering av modellen med kategorisk korsentropi som förlustfunktion och Adam som optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Sammanfattar modellens arkitektur
model.summary()


In [ ]:
# Sparar modellen så att den kan användas senare, samt tillämpar olika regulariseringsmetoder för att förbättra modellen

from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Checkpoint för att spara den bästa modellen baserat på valideringsnoggrannhet
checkpoint = ModelCheckpoint("./speechgg2lt.tf", monitor="val_accuracy", verbose=1, save_best_only=True, mode='max')

# Early stopping för att avbryta träningen när noggrannheten slutar förbättras för att undvika överträning
early_stopping = EarlyStopping(monitor='accuracy',
                               min_delta=0,
                               patience=3,
                               verbose=1,
                               restore_best_weights=True)

# Reduce learning rate on plateau för att minska inlärningshastigheten när noggrannheten slutar förbättras
reduce_learningrate = ReduceLROnPlateau(monitor='accuracy',
                                        factor=0.2,
                                        patience=3,
                                        verbose=1,
                                        min_delta=0.0001)

callbacks_list = [early_stopping, checkpoint, reduce_learningrate]


In [ ]:
# Tränar modellen med träningsdata och validerar med uppdelad träningsdata
# Använder callbacks för att spara bästa modell, avbryta vid överträning och justera inlärningshastigheten

history = model.fit(
    X, 
    y, 
    validation_split=0.2, 
    epochs=100, 
    batch_size=64, 
    shuffle=True, 
    callbacks=callbacks_list
    )

In [ ]:
## Visualiserar tränings- och valideringsförlust samt noggrannhet för att bedöma modellens prestanda och avgöra om ytterligare träning behövs

plt.style.use('dark_background')

plt.figure(figsize=(20, 10))

# Plot för tränings- och valideringsförlust
plt.subplot(1, 2, 1)
plt.title('Optimizer : Adam', fontsize=10)
plt.ylabel('Loss', fontsize=16)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend(loc='upper right')

# Plot för tränings- och valideringsnoggrannhet
plt.subplot(1, 2, 2)
plt.ylabel('Accuracy', fontsize=16)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend(loc='lower right')

plt.show()